In [ ]:
# go through every sleep event file
# add the start and end time and int value

#sort all of the data points
#fill in the -1 values

#save

In [1]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

parquetName = "whoopHypnoDf.parquet.gzip"

globalPath = "/home/chowder/Documents/personalData/Whoop/"

exportPath = "WHOOP Production-export/Health/"

files = [f for f in os.listdir(globalPath + exportPath) if f.split(".")[0][:12] == "sleep_events"]

print(len(files))

1.1.2
2


In [2]:
def whoopSleepStageToStandard(stageName):
    if stageName == "LATENCY":
        return 0
    if stageName == "WAKE":
        return 0
    if stageName == "DISTURBANCES":
        return 0
    if stageName == "LIGHT":
        return 1
    if stageName == "SWS":
        return 2
    if stageName == "REM":
        return 3
    print(stageName)
    return -1

In [3]:
files[0]

'sleep_events (1).csv'

In [10]:
import csv
with open(globalPath + exportPath + files[0]) as f:
    reader = csv.reader(f)
    lines = [x for x in reader]


[['during', 'type'],
 ["['2024-03-02T00:31:36.620Z','2024-03-02T00:37:21.728Z')", 'SWS']]

In [18]:
row = lines[1:]
row[0][0]

"['2024-03-02T00:31:36.620Z','2024-03-02T00:37:21.728Z')"

In [19]:
rows = lines[1:]
startTime = rows[0][0].split("\'")[1]
startTime

'2024-03-02T00:31:36.620Z'

In [20]:
rows = lines[1:]
endTime = rows[0][0].split("\'")[3]
endTime

'2024-03-02T00:37:21.728Z'

In [22]:
value = whoopSleepStageToStandard(rows[0][1])
value

2

In [35]:
data = []

for sf in files:
    with open(globalPath + exportPath + sf) as f:
        reader = csv.reader(f)
        lines = [x for x in reader]
        for row in lines[1:]:
            toadd = []
            toadd.append(pd.to_datetime(row[0].split("\'")[1]).round('S'))
            toadd.append(pd.to_datetime(row[0].split("\'")[3]).round('S') + datetime.timedelta(seconds=1))
            toadd.append(whoopSleepStageToStandard(row[1]))
            data.append(toadd)

print(len(data))

NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
38457


In [36]:
data = sorted(data, key=lambda x:x[0])
data[:10]

[[Timestamp('2022-04-11 08:04:24+0000', tz='UTC'),
  Timestamp('2022-04-11 08:09:14+0000', tz='UTC'),
  1],
 [Timestamp('2022-04-11 08:09:14+0000', tz='UTC'),
  Timestamp('2022-04-11 08:18:00+0000', tz='UTC'),
  0],
 [Timestamp('2022-04-11 08:18:00+0000', tz='UTC'),
  Timestamp('2022-04-11 08:19:57+0000', tz='UTC'),
  1],
 [Timestamp('2022-04-11 08:19:57+0000', tz='UTC'),
  Timestamp('2022-04-11 08:27:40+0000', tz='UTC'),
  2],
 [Timestamp('2022-04-11 08:27:40+0000', tz='UTC'),
  Timestamp('2022-04-11 08:29:36+0000', tz='UTC'),
  0],
 [Timestamp('2022-04-11 08:27:40+0000', tz='UTC'),
  Timestamp('2022-04-11 08:33:57+0000', tz='UTC'),
  1],
 [Timestamp('2022-04-11 08:33:57+0000', tz='UTC'),
  Timestamp('2022-04-11 08:54:14+0000', tz='UTC'),
  2],
 [Timestamp('2022-04-11 08:54:14+0000', tz='UTC'),
  Timestamp('2022-04-11 08:55:42+0000', tz='UTC'),
  0],
 [Timestamp('2022-04-11 08:54:14+0000', tz='UTC'),
  Timestamp('2022-04-11 09:09:12+0000', tz='UTC'),
  1],
 [Timestamp('2022-04-11 09:0

In [37]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(data)-1):
    filledRecords.append(data[ri])
    # if there is a gap between this rows end date and the next rows start date
    if data[ri][1] < data[ri+1][0]:
        filledRecords.append([
            data[ri][1],
            data[ri+1][0],
            -1
        ])

print(len(filledRecords))

48924


In [38]:
ColumnNames = ["startDate", "endDate", "value"]


whoopHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

whoopHypnoDf = whoopHypnoDf.set_index("startDate")
whoopHypnoDf

,endDate,value
startDate,,
2022-04-11 08:04:24+00:00,2022-04-11 08:09:14+00:00,1
2022-04-11 08:09:14+00:00,2022-04-11 08:18:00+00:00,0
2022-04-11 08:18:00+00:00,2022-04-11 08:19:57+00:00,1
2022-04-11 08:19:57+00:00,2022-04-11 08:27:40+00:00,2
2022-04-11 08:27:40+00:00,2022-04-11 08:29:36+00:00,0
...,...,...
2024-05-10 14:31:32+00:00,2024-05-10 14:33:27+00:00,0
2024-05-10 14:33:27+00:00,2024-05-10 14:34:25+00:00,-1
2024-05-10 14:34:25+00:00,2024-05-10 14:45:28+00:00,3


In [39]:
whoopHypnoDf.dtypes

endDate    datetime64[ns, UTC]
value                    int64
dtype: object

In [40]:
whoopHypnoDf.to_parquet(globalPath + parquetName,
              compression='gzip') 